In [ ]:

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [ ]:
MODEL = "llama3.2"

In [ ]:
# A class to represent a Webpage

class Website_scraper:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website_scraper):
    user_prompt = f"You are looking at a website titled {website_scraper.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website_scraper.text
    return user_prompt

In [ ]:
# See how this function creates exactly the format above

def prompt_payload(website_scraper):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website_scraper)}
    ]

In [ ]:
# And now: call the Ollama function instead of OpenAI

def model_response(url):
    website_scraper = Website_scraper(url)
    messages = prompt_payload(website_scraper)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [ ]:
def display_model_response(url):
    response = model_response(url)
    display(Markdown(response))

In [ ]:
display_model_response("https://revwise.com/")